패키지 설치

In [26]:
import pandas as pd
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup
import datetime as dt

날짜 구하기

In [27]:
# 오늘 날짜 구하기
today = dt.date.today()

# 이번 달 1일 날짜 구하기
start_month = dt.date(today.year, today.month, 1)

# 어제 날짜 구하기
yesterday = today - dt.timedelta(days=1)

# 1주일 전 날짜 구하기
week = today - dt.timedelta(days=7)

# 이번 달 1일 부터 어제 날짜까지 구하기
length = (yesterday - start_month).days + 1

웹 페이지 데이터 수집 코드

In [28]:
# dict 형식의 제목, 가수, 앨범, 순위 등 정보를 담을 빈 list 생성
genie_chart = []

# 날짜별 데이터 수집(이번 달 1일 부터 어제 날짜까지)
for i in range(length):
    # 날짜 계산
    date = (start_month + dt.timedelta(days=i)).strftime("%Y%m%d")
    
    # 수집할 음원 사이트의 주소 페이지 설정
    # 1~50, 51~100으로 2개의 페이지로 나눠져 있음
    pages = ['1', '2']
    for p in pages:
        
        # date = dt.datetime.now().strftime("%Y%m%d")
        
        # 수집할 컨텐츠가 있는 날짜별 웹 페이지 주소
        date = start_month.strftime("%Y%m%d")  # 이번 달 1일부터 하도록 유도
        url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh=09&rtm=N&pg={p}'
        # print(url)

        # 브라우저 버전정보
        userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

        # 접속 객체 생성
        # request는 직접 header 정보 갱신이 불가능해 session 생성
        session = requests.Session()

        # 접속 객체에 부가정보(header) 삽입, 크롤링이 가능하도록 유도
        session.headers.update({
            "Referer": "",
            "User-Agent": userAgent
        })

        # 생성한 접속 객체를 활용해 웹 페이지에 접속
        # 지정된 URL에 GET 요청을 보내고
        # 그에 대한 응답을 r 변수에 할당
        r = session.get(url)

        # 접속 실패 시 에러 코드와 에러 메시지 출력
        if r.status_code != 200:
            msg = "[%d Error] %s 에러 발생" % (r.status_code, r.reason)
            raise Exception(msg)

        # 인코딩 형식을 지정해 beautifulsoup 객체를 생성
        r.encoding = 'utf-8'

        # r 변수에 저장된 text 정보를 시각화, 확인
        # print(r.text)

        # r.text에서 HTML을 파싱하고,
        # 파싱된 HTML을 나타내는
        # BeautifulSoup 객체 soup를 생성
        soup = BeautifulSoup(r.text)
        # print(soup)



        # 추출할 음원의 수를 확인하기 위해 재생 버튼을 나타내는
        # .btn-basic.btn-listen의 수를 확인해 반복할 횟수를 정한다
        # 재생 버튼은 음원에만 존재하기 때문에 .btn-basic.btn-listen 사용
        
        genie_list = soup.select(".btn-basic.btn-listen")

        # 반복문의 길이로 사용하기 적합한지 확인
        # print(len(genie_list))
        

        # .number도 사용 가능함
        # genie_list = soup.select(".number")
        # print(len(genie_list))



        for i in range(len(genie_list)):

            # 제목 추출
            # 제목의 class는 title ellipsis였음
            songEl = soup.select('.title.ellipsis')
            song = songEl[i].text.strip()
            # print(song)
            

            # 가수 추출
            # 가수의 class는 artist ellipsis
            singerEl = soup.select('.artist.ellipsis')
            singer = singerEl[i].text.strip()
            # print(singer)
            

            # 앨범 추출
            # 앨범의 class는 albumtitle ellipsis
            albumEl = soup.select('.albumtitle.ellipsis')
            album = albumEl[i].text.strip()
            # print(album)


            # 순위 설정
            # 각 페이지 별로 순위가 다르게 입력되도록 유도
            # 1페이지는 1~50, 2페이지는 51~100으로 유도
            rank = (int(p)-1) * 50 + (i+1)


            # 추출한 데이터를 dict 형태로 변환
            item = {"가수": singer, "노래 제목": song, "앨범": album, "순위" : rank}

            # dict 정보를 genie_chart에 삽입
            genie_chart.append(item)
            
genie_chart

[{'가수': '(여자)아이들', '노래 제목': '퀸카 (Queencard)', '앨범': 'I feel', '순위': 1},
 {'가수': 'IVE (아이브)', '노래 제목': 'I AM', '앨범': "I've IVE", '순위': 2},
 {'가수': 'aespa',
  '노래 제목': 'Spicy',
  '앨범': 'MY WORLD - The 3rd Mini Album',
  '순위': 3},
 {'가수': 'IVE (아이브)', '노래 제목': 'Kitsch', '앨범': "I've IVE", '순위': 4},
 {'가수': 'LE SSERAFIM (르세라핌)',
  '노래 제목': 'UNFORGIVEN (Feat. Nile Rodgers)',
  '앨범': 'UNFORGIVEN',
  '순위': 5},
 {'가수': '지수 (JISOO)', '노래 제목': '꽃', '앨범': 'ME', '순위': 6},
 {'가수': 'NewJeans',
  '노래 제목': 'Hype boy',
  '앨범': "NewJeans 1st EP 'New Jeans'",
  '순위': 7},
 {'가수': 'NewJeans', '노래 제목': 'Ditto', '앨범': "NewJeans 'OMG'", '순위': 8},
 {'가수': '윤하 (YOUNHA)',
  '노래 제목': '사건의 지평선',
  '앨범': "YOUNHA 6th Album Repackage 'END THEORY : Final Edition'",
  '순위': 9},
 {'가수': '세븐틴 (SEVENTEEN)',
  '노래 제목': '손오공',
  '앨범': "SEVENTEEN 10th Mini Album 'FML'",
  '순위': 10},
 {'가수': 'FIFTY FIFTY',
  '노래 제목': 'Cupid',
  '앨범': 'The Beginning : Cupid',
  '순위': 11},
 {'가수': 'Charlie Puth',
  '노래 제목': "I Don't Think That I

DataFrame 사용

In [29]:
df = DataFrame(genie_chart)

# 순위를 우선해서 보여주기 위해 index로 설정
df.set_index('순위',inplace=True)
# df
df.to_excel("확인용.xlsx")